# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

In [2]:
# Loading the csv exported in WeatherPy to a DataFrame
csv_to_load="../WeatherPy/WeatherInCities.csv"
Weather_df=pd.read_csv(csv_to_load)
Weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Max Temp,Lat,Lng,Wind Speed
0,0,Lindi,20,TZ,1587653758,70,29.00,-10.00,39.72,5.10
1,1,Hobart,40,AU,1587653759,71,13.33,-42.88,147.33,2.60
2,2,Hermanus,83,ZA,1587653759,65,20.56,-34.42,19.23,3.13
3,3,Treinta y Tres,91,UY,1587653759,75,21.32,-33.23,-54.38,2.58
4,4,Nayoro,100,JP,1587653759,95,-2.40,44.35,142.46,1.88
...,...,...,...,...,...,...,...,...,...,...
563,563,Mogadishu,79,SO,1587653849,71,29.09,2.04,45.34,3.65
564,564,Sorong,94,ID,1587653662,88,25.24,-0.88,131.25,0.48
565,565,Udachny,96,RU,1587653850,94,0.00,66.42,112.40,3.00
566,566,Kirakira,100,SB,1587653850,78,26.34,-10.45,161.92,1.35


In [3]:
#HUMIDITY HEATMAP

# Configure gmaps
gmaps.configure(api_key=" ")

# Store latitude and longitude in locations
locations = Weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
Humidity = Weather_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Creating new DataFrame fitting weather criteria

In [5]:
Vacation_df=Weather_df.loc[((Weather_df['Max Temp']<60)&
                           (Weather_df['Max Temp']>12)&
                           (Weather_df['Wind Speed']<20)&
                           (Weather_df['Cloudiness']==0))]
Vacation_df=Vacation_df.dropna(how='any')
Vacation_df.count()

Unnamed: 0    39
City          39
Cloudiness    39
Country       39
Date          39
Humidity      39
Max Temp      39
Lat           39
Lng           39
Wind Speed    39
dtype: int64

In [6]:
#Adding list of hotels wrt to the cordinates

hotel_df=Vacation_df.copy()
hotel_df['Hotel Name']=''
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Max Temp,Lat,Lng,Wind Speed,Hotel Name
45,45,Isabela,0,PH,1587653767,76,27.11,6.71,121.97,0.68,
55,55,Esperance,0,AU,1587653769,93,15.00,-33.87,121.90,3.10,
72,72,Castro,0,BR,1587653772,24,22.40,-24.79,-50.01,1.88,
74,74,San Matías,0,BO,1587653772,44,31.86,-16.37,-58.40,1.82,
139,139,Nouadhibou,0,MR,1587653784,68,21.00,20.93,-17.03,9.80,


In [7]:
#tesing with one location
params = {"location":"20.52,-13.05",
          "radius":5000,
          "types":"lodging",
           "key":" "}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
response=requests.get(base_url,params=params).json()
print(json.dumps(response, indent=4,sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 20.5224272,
                    "lng": -13.0503926
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.5238127302915,
                        "lng": -13.0490953697085
                    },
                    "southwest": {
                        "lat": 20.5211147697085,
                        "lng": -13.0517933302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "96fe7ef436c7fef3339fac83cd7f5ece884b456c",
            "name": "Oumou Elghoura",
            "photos": [
                {
                    "height": 2304,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/106460871107768605214\">Pepe atar<

In [10]:
# create a params dict that will be updated with new city each iteration
params = {"radius":5000,
          "types":"lodging",
           "key": " "}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    lat=hotel_df['Lat']
    lng=hotel_df['Lng']
     
    # update address key value
    params['location'] = f"{lat[index]},{lng[index]}"
    response=requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
       
    except (KeyError, IndexError):
        print("Missing Hotel... skipping.")

Missing Hotel... skipping.
Missing Hotel... skipping.


In [11]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Max Temp,Lat,Lng,Wind Speed,Hotel Name
45,45,Isabela,0,PH,1587653767,76,27.11,6.71,121.97,0.68,Isawad Resort
55,55,Esperance,0,AU,1587653769,93,15.00,-33.87,121.90,3.10,Hospitality Esperance
72,72,Castro,0,BR,1587653772,24,22.40,-24.79,-50.01,1.88,CHACARA BAILLY
74,74,San Matías,0,BO,1587653772,44,31.86,-16.37,-58.40,1.82,"HOTEL ""SAN JOSE"""
139,139,Nouadhibou,0,MR,1587653784,68,21.00,20.93,-17.03,9.80,El Medina
140,140,Cidreira,0,BR,1587653784,62,23.44,-30.18,-50.21,2.78,Novomar Pousada e Turismo Ltda
151,151,Aswān,0,EG,1587653786,8,36.81,24.09,32.91,5.06,Nuba Nile
158,158,Kumul,0,CN,1587653787,13,18.46,42.80,93.45,5.00,Laoyutang Resort
209,209,San Quintín,0,MX,1587653796,57,20.01,30.48,-115.95,1.41,Old Mill Hotel
244,244,Province of Mantua,0,IT,1587653802,26,22.78,45.17,10.78,2.10,Hotel la Favorita


In [14]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))